# Import Libraries

In [1]:
# Libraries
import pandas as pd
import numpy as np
import requests 
import re
import joblib

# Selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# BeautifulSoup
from bs4 import BeautifulSoup as bs

# Permanently changes the pandas display settings
pd.set_option('display.max_columns', None)

# Web Scrape

In [205]:
# Launch Browser 
# Setup the Selenium WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

In [228]:
# Initialize DataFrame
# Specify column names to use with dataframe
columns = ['School Name', 'Full Address', 'Street Address', 'City', 'State', 'Zip Code', 'Country', 'School Type', 'Website', 'Description']

# Initialize DataFrame
df = pd.DataFrame(columns=columns)

print(df)

Empty DataFrame
Columns: [School Name, Full Address, Street Address, City, State, Zip Code, Country, School Type, Website, Description]
Index: []


In [229]:
# Scrape data from each state/province page 
# Create list of States & Provinces in the USA & Canada to loop over
state_province = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'AB', 'BC', 'MB', 'NB', 'NL', 'NS', 'ON', 'PE', 'QC', 'SK']
province = ['AB', 'BC', 'MB', 'NB', 'NL', 'NS', 'ON', 'PE', 'QC', 'SK']

for state in state_province:

    if state in province: 
        co_code = 'CA'

    else:
        co_code = 'US'

    # Create link for each state/province 
    url = f'https://www.adventisteducation.org/schools?stateProvince={state}%3B{co_code}'

    # Open page
    driver.get(url)

    # Wait for the element to become clickable
    element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//*[contains(@class, 'Schools_searchResults__')]"))
    )
    # Once the element is clickable, get the outer HTML
    html_content = element.get_attribute("outerHTML")

    # Use BeautifulSoup to parse the HTML content
    soup = bs(html_content, "html.parser")



    # Find each 'a' anchor tag matching the regex search below to find info for each school
    result_divs = soup.find_all('a', class_ = re.compile('SearchResult_root__'))

    # Loop over each result to extract name of school
    for div in result_divs:


        # School Name
        # Find name of school 
        name_heading = div.find_all('h2', class_ = re.compile('SearchResult_heading__'))
        name_pattern = r">.*?</"

        # Use regex to find the pattern in the name_heading and extract the matching text
        name = re.search(name_pattern, str(name_heading))
        
        # Extract all matches in a single group
        name = name.group()
        
        # Use string slicing to remove the ends that helped return the result
        name = name[1:-2]


        # School Address
        # Find & Collect each full address
        address_heading = div.find_all('div', class_ = re.compile('SearchResult_address__'))
        address_pattern  = r">.*?</"
        address_full = re.search(address_pattern, str(address_heading))
        address_full = address_full.group()

        # Trim result address string of regex remnants
        address_full = address_full[1:-2]


        try:
            # Extract different parts of address information
            address_detail_pattern  = re.compile(r"(.*(\s.*)*,)\s(.*,)\s(\w*,)\s(\d+-*\d*,|.{3}\s.{3},)\s(\w*)")  # regex expression
            address_detail = address_detail_pattern.search(address_full)   # search full address for patterns

            if address_detail != None:
                # Save each section of results to a different variable
                address_street, address_road_type, address_city, address_state, address_zip, address_country = address_detail.groups()  # extract each group from pattern

                # Use string slicing to remove the ends that helped return the result
                address_street = address_street[:-1]
                address_city = address_city[:-1]
                address_state = address_state[:-1]
                address_zip = address_zip[:-1]

            else: 
                # If there is no match, set default value
                address_street, address_road_type, address_city, address_state, address_zip, address_country = None

        
        except Exception as e:
            # Set a default value if any exceptions occur during regex search
            print(f"An error occurred: {e}")
            address_street, address_road_type, address_city, address_state, address_zip, address_country = None
        


        # School Type
        # Find school type
        school_heading = div.find_all('dd')

        # Extract school type
        school_pattern  = re.compile(r">(.*)</")  # regex expression
        school_type = re.search(school_pattern, str(school_heading))  # search for pattern
        school_type = school_type.group()   # extract result from group 

        school_type = school_type[1:-2]


        # School Website
        # Find link
        link_pattern  = re.compile('href="/schools/(.){6}"')
        link_partial = re.search(link_pattern, str(div))
        link_partial = link_partial.group()

        link_partial = link_partial[6:-1]

        link_root = 'https://www.adventisteducation.org'

        link_full = link_root + link_partial


        # School Description
        # Find description
        desc_full = div.find_all('div', class_ = re.compile('SearchResult_intro__'))
        
        # Check if there is a description or not
        if desc_full:

            # Convert matching result to a string and trim off excess
            desc_full = str(desc_full)
            desc_full = desc_full[40:-7]


        new_row = pd.DataFrame([{'School Name': name, 'Full Address': address_full, 'Street Address': address_street, 'City': address_city, 'State': address_state, 'Zip Code': address_zip, 'Country': address_country, 'School Type': school_type, 'Website': link_full, 'Description': desc_full}])
        #print(new_row)

        df = pd.concat([df, new_row], ignore_index=True)

display(df)




,School Name,Full Address,Street Address,City,State,Zip Code,Country,School Type,Website,Description
0,Bethany Christian Academy,"1765 Highland Ave, Montgomery, AL, 36107-2657,...",1765 Highland Ave,Montgomery,AL,36107-2657,USA,PK-08,https://www.adventisteducation.org/schools/ANTP25,Bethany Christian Academy is a Christ-centered...
1,Bethany SDA Child Development Center,"714 Cedar St, Montgomery, AL, 36106-1002, USA",714 Cedar St,Montgomery,AL,36106-1002,USA,ECP,https://www.adventisteducation.org/schools/ANTP27,[]
2,Big Cove Christian Academy,"6354 Highway 431 S, Owens Cross Roads, AL, 357...",6354 Highway 431 S,Owens Cross Roads,AL,35763-9210,USA,PK-08,https://www.adventisteducation.org/schools/ANTG16,[]
3,Ephesus Academy Child Development Center,"829 McMillon Ave SW, Birmingham, AL, 35211-173...",829 McMillon Ave SW,Birmingham,AL,35211-1730,USA,ECP,https://www.adventisteducation.org/schools/ANTP2A,[]
4,Ephesus Junior Academy,"829 McMillon Ave SW, Birmingham, AL, 35211-173...",829 McMillon Ave SW,Birmingham,AL,35211-1730,USA,PK-10,https://www.adventisteducation.org/schools/ANTP45,[]
...,...,...,...,...,...,...,...,...,...,...
885,Windsor Adventist Elementary School,"5350 Haig Ave, Windsor, ON, N8T 1K8, Canada",5350 Haig Ave,Windsor,ON,N8T 1K8,Canada,PK-08,https://www.adventisteducation.org/schools/AN6M90,[]
886,Greaves Adventist Academy,"2330 West Hill Ave, Montreal, QC, H4B 2S4, Canada",2330 West Hill Ave,Montreal,QC,H4B 2S4,Canada,PK-12,https://www.adventisteducation.org/schools/AN6P15,[]
887,Sartigan Adventist Academy,"645 7e Rue Sartigan, Saint-Georges, QC, G5Y 5B...",645 7e Rue Sartigan,Saint-Georges,QC,G5Y 5B8,Canada,PK-08,https://www.adventisteducation.org/schools/AN6PAS,[]
888,Curtis-Horne Christian School,"3718 Hill Ave, Regina, SK, S4S 0X5, Canada",3718 Hill Ave,Regina,SK,S4S 0X5,Canada,K-09,https://www.adventisteducation.org/schools/AN6B20,[]


In [231]:
# Remember to close the browser
driver.quit()

# Verify Street Address

In [236]:
# Verify if each street address was correctly extracted from the full address
df['Addresses Match'] = df.apply(lambda row: row['Full Address'].startswith(row['Street Address']), axis=1)

In [240]:
df.head(1)

,School Name,Full Address,Street Address,City,State,Zip Code,Country,School Type,Website,Description,Addresses Match
0,Bethany Christian Academy,"1765 Highland Ave, Montgomery, AL, 36107-2657,...",1765 Highland Ave,Montgomery,AL,36107-2657,USA,PK-08,https://www.adventisteducation.org/schools/ANTP25,Bethany Christian Academy is a Christ-centered...,True


In [238]:
# Count the number of False values returned (a mismatch between address values)
df['Addresses Match'].value_counts()

True    890
Name: Addresses Match, dtype: int64

We can see that there are zero False values in the 'Addresses Match' column. There should be no address mismatches between the 'Full Address' and 'Street Address', which was an issue earlier in the development. 

As such, we will drop the new column as it is no longer needed.

In [241]:
df.drop('Addresses Match', axis=1, inplace=True)
df.head(1)

,School Name,Full Address,Street Address,City,State,Zip Code,Country,School Type,Website,Description
0,Bethany Christian Academy,"1765 Highland Ave, Montgomery, AL, 36107-2657,...",1765 Highland Ave,Montgomery,AL,36107-2657,USA,PK-08,https://www.adventisteducation.org/schools/ANTP25,Bethany Christian Academy is a Christ-centered...


# Export Results to File

In [230]:
# Export dataframe to a CSV file

# Specify file name and include datetime index column for CSV file 
df.to_csv('Data/2024_NASDA_Education_processed.csv', index=True)

# Serialize Results to File

Use joblib to serialize data and variables.

In [235]:
# serialize results to a file: future

# create dictionary with variables to save for later
export = {
    'df': df
    }

# save dictionary as a file
joblib.dump(export, '2024_NASDA_Education_export')

# import joblib file with serialized variables
loaded_data = joblib.load('2024_NASDA_Education_export')

# display keys to use to save to new/active variables
loaded_data.keys()

dict_keys(['df'])

# Summary

In this project, the goal of consolidating the different types and locations of Adventist schools across North America was achieved via the official Adventist Education website. 

The consolidated information will be made into a dynamic map using Tableau so that people considering moving in North America can find, *in one place*, what Adventist Education options are available where they might be relocating to.

# Geocoding & Grade Filtering

After importing the exported file above into Tableau, a few short-comings were noticed and will need some more processing in Python before being imported back into Tableau again:
> * Tableau is unable to plot street addresses on a map without having said addresses Geocoded into Latitude & Longitude coordinates.
> * Tableau is also unable to create a filter by specific grades from the Shool Type string (i.e. "PK-12"). Grades will need to be processed (One-Hot Encoded) to make filtering straightforward and reliable. 

# Load the New Data

In [2]:
# Import the data
path = 'Data/2024_NASDA_Education_processed_geocodio.csv'

df_OE = pd.read_csv(path)

In [3]:
# Display first few rows of imported file
# Geocoding was provided by https://dash.geocod.io which added the additional rows at the end to the original file.
df_OE.head(3)

,Unnamed: 0,School Name,Full Address,Street Address,City,State,Zip Code,Country,School Type,Website,Description,Latitude,Longitude,Accuracy Score,Accuracy Type,Number,Street,Unit Type,Unit Number,City.1,State.1,County,Zip,Country.1,Source
0,0,Bethany Christian Academy,"1765 Highland Ave, Montgomery, AL, 36107-2657,...",1765 Highland Ave,Montgomery,AL,36107-2657,USA,PK-08,https://www.adventisteducation.org/schools/ANTP25,Bethany Christian Academy is a Christ-centered...,32.372560,-86.284475,1.0,rooftop,1765,Highland Ave,NaN,NaN,Montgomery,AL,Montgomery County,36107,US,City of Montgomery
1,1,Bethany SDA Child Development Center,"714 Cedar St, Montgomery, AL, 36106-1002, USA",714 Cedar St,Montgomery,AL,36106-1002,USA,ECP,https://www.adventisteducation.org/schools/ANTP27,[],32.371569,-86.289672,1.0,rooftop,714,Cedar St,NaN,NaN,Montgomery,AL,Montgomery County,36106,US,City of Montgomery
2,2,Big Cove Christian Academy,"6354 Highway 431 S, Owens Cross Roads, AL, 357...",6354 Highway 431 S,Owens Cross Roads,AL,35763-9210,USA,PK-08,https://www.adventisteducation.org/schools/ANTG16,[],34.667724,-86.488204,1.0,range_interpolation,6354,US-431,NaN,NaN,Owens Cross Roads,AL,Madison County,35763,US,TIGER/Line® dataset from the US Census Bureau


In [20]:
# create copy in memory so dataframe does not need to be reimported to rerun cells below
dfx = df_OE.copy()

In [21]:
# verify column names and data types
dfx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 890 entries, 0 to 889
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      890 non-null    int64  
 1   School Name     890 non-null    object 
 2   Full Address    890 non-null    object 
 3   Street Address  890 non-null    object 
 4   City            890 non-null    object 
 5   State           890 non-null    object 
 6   Zip Code        890 non-null    object 
 7   Country         890 non-null    object 
 8   School Type     890 non-null    object 
 9   Website         890 non-null    object 
 10  Description     890 non-null    object 
 11  Latitude        890 non-null    float64
 12  Longitude       890 non-null    float64
 13  Accuracy Score  890 non-null    float64
 14  Accuracy Type   890 non-null    object 
 15  Number          867 non-null    object 
 16  Street          878 non-null    object 
 17  Unit Type       9 non-null      obj

# Clean Data

## Rename Features for Clarity

In [22]:
dfx.head(1)

,Unnamed: 0,School Name,Full Address,Street Address,City,State,Zip Code,Country,School Type,Website,Description,Latitude,Longitude,Accuracy Score,Accuracy Type,Number,Street,Unit Type,Unit Number,City.1,State.1,County,Zip,Country.1,Source
0,0,Bethany Christian Academy,"1765 Highland Ave, Montgomery, AL, 36107-2657,...",1765 Highland Ave,Montgomery,AL,36107-2657,USA,PK-08,https://www.adventisteducation.org/schools/ANTP25,Bethany Christian Academy is a Christ-centered...,32.37256,-86.284475,1.0,rooftop,1765,Highland Ave,NaN,NaN,Montgomery,AL,Montgomery County,36107,US,City of Montgomery


In [23]:
# Keep both Zip Code and Zip, but rename to clarify between the number of digits included.
dfx = dfx.rename(columns={"Zip Code": "Zip (full)", "Zip": "Zip (5-digit)"})
dfx.head(1)

,Unnamed: 0,School Name,Full Address,Street Address,City,State,Zip (full),Country,School Type,Website,Description,Latitude,Longitude,Accuracy Score,Accuracy Type,Number,Street,Unit Type,Unit Number,City.1,State.1,County,Zip (5-digit),Country.1,Source
0,0,Bethany Christian Academy,"1765 Highland Ave, Montgomery, AL, 36107-2657,...",1765 Highland Ave,Montgomery,AL,36107-2657,USA,PK-08,https://www.adventisteducation.org/schools/ANTP25,Bethany Christian Academy is a Christ-centered...,32.37256,-86.284475,1.0,rooftop,1765,Highland Ave,NaN,NaN,Montgomery,AL,Montgomery County,36107,US,City of Montgomery


In [24]:
# Keep both Number, but rename to show that it is part of the separated items (Street & Street Number) from what was originally combined in Street Address
dfx = dfx.rename(columns={"Number": "Street Number"})
dfx.head(1)

,Unnamed: 0,School Name,Full Address,Street Address,City,State,Zip (full),Country,School Type,Website,Description,Latitude,Longitude,Accuracy Score,Accuracy Type,Street Number,Street,Unit Type,Unit Number,City.1,State.1,County,Zip (5-digit),Country.1,Source
0,0,Bethany Christian Academy,"1765 Highland Ave, Montgomery, AL, 36107-2657,...",1765 Highland Ave,Montgomery,AL,36107-2657,USA,PK-08,https://www.adventisteducation.org/schools/ANTP25,Bethany Christian Academy is a Christ-centered...,32.37256,-86.284475,1.0,rooftop,1765,Highland Ave,NaN,NaN,Montgomery,AL,Montgomery County,36107,US,City of Montgomery


## Drop Redundant or Unneeded Features

In [25]:
# Create an empty list of columns to be dropped
to_drop = []

# Add previous index row to the drop list
to_drop.append("Unnamed: 0")

# Add duplicated and/or not needed features added by geocod.io to the drop list
to_drop.extend(["Accuracy Score", "Accuracy Type", "Unit Type", "Unit Number", "City.1", "State.1", "Country.1", "Source"])

print(to_drop)

['Unnamed: 0', 'Accuracy Score', 'Accuracy Type', 'Unit Type', 'Unit Number', 'City.1', 'State.1', 'Country.1', 'Source']


In [26]:
# Drop specified columns
dfx = dfx.drop(columns=to_drop, axis=1)
dfx.head(1)

,School Name,Full Address,Street Address,City,State,Zip (full),Country,School Type,Website,Description,Latitude,Longitude,Street Number,Street,County,Zip (5-digit)
0,Bethany Christian Academy,"1765 Highland Ave, Montgomery, AL, 36107-2657,...",1765 Highland Ave,Montgomery,AL,36107-2657,USA,PK-08,https://www.adventisteducation.org/schools/ANTP25,Bethany Christian Academy is a Christ-centered...,32.37256,-86.284475,1765,Highland Ave,Montgomery County,36107


## Reorder the Columns

In [27]:
# Create a list that will hold the names of each column, in the new desired order.
new_order = ["School Name", "Full Address", "Street Address", "Street Number", "Street", "City", "County", "State", "Zip (5-digit)", "Zip (full)", "Country", "School Type", "Website", "Description", "Latitude", "Longitude"]

# Rearrange the order of columns using .reindex()
dfx = dfx.reindex(columns=new_order)

display(dfx.head(1))

,School Name,Full Address,Street Address,Street Number,Street,City,County,State,Zip (5-digit),Zip (full),Country,School Type,Website,Description,Latitude,Longitude
0,Bethany Christian Academy,"1765 Highland Ave, Montgomery, AL, 36107-2657,...",1765 Highland Ave,1765,Highland Ave,Montgomery,Montgomery County,AL,36107,36107-2657,USA,PK-08,https://www.adventisteducation.org/schools/ANTP25,Bethany Christian Academy is a Christ-centered...,32.37256,-86.284475


## Fix Description when No Value

In [28]:
# Description currently has "[]" as the "null" value and diplays poorly in Tableau
# Replace "null" value with an empty string "" to improve its display in Tableau

# Count number of instances, currently
print(dfx["Description"].value_counts().head(1))  # 606 instances of "[]"

# Replacement
dfx["Description"] = dfx["Description"].replace("[]", "")

# Recount updated number of instances
print(dfx["Description"].value_counts().head(1))   # 606 instances of ""

[]    606
Name: Description, dtype: int64
    606
Name: Description, dtype: int64


## Fix JP-12 Value

In [29]:
# There is a single value that was found through exploring the data in Tableau that appears to be either a typo or an outlier. 
# We will change this value to a more common value that is either very similar or what it actually should have been, originally.

# in School Type, explore the value "JP-12" 
print(dfx['School Type'].value_counts().sort_values().head(7))  # JK is a much more common value and is similar or probably correct
print()

# Replacement
dfx['School Type'] = dfx['School Type'].replace('JP-12', 'JK-12')

# Verify update
print(dfx['School Type'].value_counts().sort_values().head(6))  # Successfully updated

10-12      1
K-8        1
JP-12      1
JK-12      1
College    3
K-10       6
JK-08      6
Name: School Type, dtype: int64

10-12      1
K-8        1
JK-12      2
College    3
K-10       6
JK-08      6
Name: School Type, dtype: int64


## Fix Values Without a Leading Zero
Two values do not have a leading zero in the single-digit number:
> * "K-8"
> * "9-12"
We will add a leading zero to each for consistency.

In [38]:
# Explore the value "K-8" 
print(dfx['School Type'].value_counts().sort_values())  
print()

# Replacement
dfx['School Type'] = dfx['School Type'].replace('K-8', 'K-08')

# Verify update
print(dfx['School Type'].value_counts())  # Successfully updated

10-12           1
K-8             1
JK-12           2
College         3
K-10            6
JK-08           6
University     11
K-09           14
K-12           30
PK-12          34
9-12           44
PK-10          69
K-08           76
ECP           169
PK-08         424
Name: School Type, dtype: int64

PK-08         424
ECP           169
K-08           77
PK-10          69
9-12           44
PK-12          34
K-12           30
K-09           14
University     11
K-10            6
JK-08           6
College         3
JK-12           2
10-12           1
Name: School Type, dtype: int64


In [39]:
# Explore the value "9-12" 
print(dfx['School Type'].value_counts().sort_values())  
print()

# Replacement
dfx['School Type'] = dfx['School Type'].replace('9-12', '09-12')

# Verify update
print(dfx['School Type'].value_counts())  # Successfully updated

10-12           1
JK-12           2
College         3
K-10            6
JK-08           6
University     11
K-09           14
K-12           30
PK-12          34
9-12           44
PK-10          69
K-08           77
ECP           169
PK-08         424
Name: School Type, dtype: int64

PK-08         424
ECP           169
K-08           77
PK-10          69
09-12          44
PK-12          34
K-12           30
K-09           14
University     11
K-10            6
JK-08           6
College         3
JK-12           2
10-12           1
Name: School Type, dtype: int64


# Grade Filtering
It is currently impossible to filter by grade level in Tableau using the School Type text field (i.e. - "PK-08", "K-12", "ECP", etc). We will use One-Hot Encoding to specifically specify which grades are available at each school, preschool through the last year of high school (or college/university). 

In [41]:
school_type_data = {'School Type': ['10-12', '09-12', 'College', 'ECP', 'JK-08', 'JK-12', 'K-08', 'K-09', 'K-10', 'K-12', 'PK-08', 'PK-10', 'PK-12', 'University']}
school_types = pd.DataFrame(school_type_data)

display(school_types)

,School Type
0,10-12
1,09-12
2,College
3,ECP
4,JK-08
5,JK-12
6,K-08
7,K-09
8,K-10
9,K-12


In [42]:
# 
preschool = ['J', 'P']
school_types['Preschool'] = school_types['School Type'].str.contains('|'.join(preschool))
print(school_types)  # correctly returns only preschool grades

   School Type  Preschool
0        10-12      False
1        09-12      False
2      College      False
3          ECP       True
4        JK-08       True
5        JK-12       True
6         K-08      False
7         K-09      False
8         K-10      False
9         K-12      False
10       PK-08       True
11       PK-10       True
12       PK-12       True
13  University      False


In [ ]:
'''
INSTEAD, do One-Hot Encoding in one, fell swoop.
will need to manually type out a list-like string, separated by commas for each School Type above

Then split with the code below.
will want to keep a list-like string, maybe an actual list, the OHE columns (for whatever ends up working the best in Tableau)

'''

'''
import pandas as pd

# Example dataset
data = {
    'School': ['School A', 'School B', 'School C'],
    'Grades': ['1,2,3', '1,2,3,4,5', 'Kindergarten,1']
}

df = pd.DataFrame(data)

# Convert 'Grades' column to one-hot encoding
grades_expanded = df['Grades'].str.get_dummies(sep=',')

# Join the expanded grades back to the original dataframe
df_encoded = df.join(grades_expanded)

print(df_encoded)

'''